In [31]:
import json
import time
import webbrowser
import pandas as pd
from pandas.io.json import json_normalize
from rauth import OAuth2Service
from rauth.utils import parse_utf8_qsl
import base64

In [32]:
credentials_file = open('../oauth.json')
credentials = json.load(credentials_file)
credentials_file.close()

In [44]:
service_params = {'client_id':credentials['consumer_key'],
                  'client_secret':credentials['consumer_secret'],
                  'name':'yahoo',
                  'access_token_url':'https://api.login.yahoo.com/oauth2/get_token',
                  'authorize_url':'https://api.login.yahoo.com/oauth2/request_auth',
                  'base_url':'http://fantasysports.yahooapis.com/'
                 }

In [45]:
oauth = OAuth2Service(**service_params)

In [46]:
authorize_url = oauth.get_authorize_url(client_secret=credentials['consumer_secret'], 
                                        redirect_uri='oob', 
                                        response_type='code')

In [47]:
webbrowser.open(authorize_url)

True

In [48]:
verify = input('Enter code: ')

Enter code: aey58nq


In [49]:
def generate_oauth2_headers():
        """Generates header for oauth2
        """
        encoded_credentials = base64.b64encode(('{0}:{1}'.format(credentials['consumer_key'],
                                                                 credentials['consumer_secret'])).encode('utf-8'))
        headers={
            'Authorization':'Basic {0}'.format(encoded_credentials.decode('utf-8')),
            'Content-Type': 'application/x-www-form-urlencoded'
        }

        return headers

In [50]:
def oauth2_access_parser(raw_access):
        """Parse oauth2 access
        """
        parsed_access = json.loads(raw_access.content.decode('utf-8'))
        access_token = parsed_access['access_token']
        token_type = parsed_access['token_type']
        refresh_token = parsed_access['refresh_token']
        guid = parsed_access['xoauth_yahoo_guid']

        keys = {
            'access_token': access_token,
            'token_type': token_type,
            'refresh_token': refresh_token,
            'guid': guid
        }
        
        return keys
        


In [51]:
token_time = time.time()
    
keys = {'token_time': token_time}

headers = generate_oauth2_headers()
# Getting access token
raw_access = oauth.get_raw_access_token(data={"code": verify, 'redirect_uri': 'oob','grant_type':'authorization_code'}, headers=headers)
#parsed_access = parse_utf8_qsl(raw_access.content.decode('utf-8'))
keys.update(oauth2_access_parser(raw_access))

keys.update(service_params)

In [52]:
with open('secrets.json', 'w') as fp:
    json.dump(keys, fp, indent=4, sort_keys=True, ensure_ascii=False)

In [57]:
s = oauth.get_session(token=keys['access_token'])

### Get league settings and user team

In [65]:
url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nba.l.171968/settings"
params = {'format': 'json'}
response = s.get(url, params=params)

In [71]:
response_content = response.json()
num_teams = response_content['fantasy_content']['league'][0]['num_teams']

12

In [84]:
df_positions = pd.DataFrame(list(map(lambda x: x['roster_position'], 
    response_content['fantasy_content']['league'][1]['settings'][0]['roster_positions'])))

In [92]:
stats = [i['stat']['name']\
 for i in response_content['fantasy_content']['league'][1]['settings'][0]['stat_categories']['stats'] \
 if 'is_only_display_stat' not in i['stat']['stat_position_types'][0]['stat_position_type'].keys()]